In [1]:
import pandas as pd


In [2]:

import numpy as np

In [3]:
import matplotlib.pyplot as plt

In [4]:
from mlxtend.frequent_patterns import apriori,association_rules


In [5]:
df=pd.read_excel('Online_Retail.xlsx')

In [6]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-01-12 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-01-12 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-09-12 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-09-12 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-09-12 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-09-12 12:50:00,4.15,12680.0,France


In [20]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,540454.000000,540454.000000,406829.000000
mean,9.603887,4.623528,15287.690570
std,218.007088,96.889717,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [7]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [8]:
df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

## Data preprocessing

### 1. Removing the invoice date columns



In [9]:
df=df.drop('InvoiceDate',axis=1)

### 2. Removing all Missing values from Description cols

In [12]:
df['Description'].isna().sum()
df.dropna(subset=['Description'],axis=0,inplace=True)
df['Description'].isna().sum()

0

###  3. Removing white spaces if any in Description column



In [13]:
df['Description']=df['Description'].str.strip()
df.dropna(subset=['Description'],axis=0,inplace=True)
df['Description'].isna().sum()


0

### 4.changing InvoiceNo column datatype to str

In [16]:
df['InvoiceNo']=df['InvoiceNo'].astype('str')


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 540454 entries, 0 to 541908
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    540454 non-null  object 
 1   StockCode    540454 non-null  object 
 2   Description  540454 non-null  object 
 3   Quantity     540454 non-null  int64  
 4   UnitPrice    540454 non-null  float64
 5   CustomerID   406829 non-null  float64
 6   Country      540454 non-null  object 
dtypes: float64(2), int64(1), object(4)
memory usage: 33.0+ MB


## High utility items(no. of times the item is sold in diff invoices

In [21]:
list1=df['Description']


In [22]:
list1

0          WHITE HANGING HEART T-LIGHT HOLDER
1                         WHITE METAL LANTERN
2              CREAM CUPID HEARTS COAT HANGER
3         KNITTED UNION FLAG HOT WATER BOTTLE
4              RED WOOLLY HOTTIE WHITE HEART.
                         ...                 
541904            PACK OF 20 SPACEBOY NAPKINS
541905            CHILDREN'S APRON DOLLY GIRL
541906           CHILDRENS CUTLERY DOLLY GIRL
541907        CHILDRENS CUTLERY CIRCUS PARADE
541908           BAKING SET 9 PIECE RETROSPOT
Name: Description, Length: 540454, dtype: object

In [23]:
from collections import Counter
dict1=Counter(list1)


In [24]:
dict1

Counter({'WHITE HANGING HEART T-LIGHT HOLDER': 2369,
         'WHITE METAL LANTERN': 328,
         'CREAM CUPID HEARTS COAT HANGER': 293,
         'KNITTED UNION FLAG HOT WATER BOTTLE': 473,
         'RED WOOLLY HOTTIE WHITE HEART.': 449,
         'SET 7 BABUSHKA NESTING BOXES': 389,
         'GLASS STAR FROSTED T-LIGHT HOLDER': 141,
         'HAND WARMER UNION JACK': 515,
         'HAND WARMER RED POLKA DOT': 18,
         'ASSORTED COLOUR BIRD ORNAMENT': 1501,
         "POPPY'S PLAYHOUSE BEDROOM": 436,
         "POPPY'S PLAYHOUSE KITCHEN": 451,
         'FELTCRAFT PRINCESS CHARLOTTE DOLL': 469,
         'IVORY KNITTED MUG COSY': 111,
         'BOX OF 6 ASSORTED COLOUR TEASPOONS': 213,
         'BOX OF VINTAGE JIGSAW BLOCKS': 243,
         'BOX OF VINTAGE ALPHABET BLOCKS': 272,
         'HOME BUILDING BLOCK WORD': 806,
         'LOVE BUILDING BLOCK WORD': 656,
         'RECIPE BOX WITH METAL HEART': 8,
         'DOORMAT NEW ENGLAND': 608,
         'JAM MAKING SET WITH JARS': 1229,
    

In [25]:
high_u_items=sorted(dict1.items(),key=lambda x:x[1],reverse=True)


In [26]:
high_u_items

[('WHITE HANGING HEART T-LIGHT HOLDER', 2369),
 ('REGENCY CAKESTAND 3 TIER', 2200),
 ('JUMBO BAG RED RETROSPOT', 2159),
 ('PARTY BUNTING', 1727),
 ('LUNCH BAG RED RETROSPOT', 1638),
 ('ASSORTED COLOUR BIRD ORNAMENT', 1501),
 ('SET OF 3 CAKE TINS PANTRY DESIGN', 1473),
 ('PACK OF 72 RETROSPOT CAKE CASES', 1385),
 ('LUNCH BAG  BLACK SKULL.', 1350),
 ('NATURAL SLATE HEART CHALKBOARD', 1280),
 ('POSTAGE', 1252),
 ('JUMBO BAG PINK POLKADOT', 1251),
 ('HEART OF WICKER SMALL', 1237),
 ('JAM MAKING SET WITH JARS', 1229),
 ('JUMBO STORAGE BAG SUKI', 1214),
 ("PAPER CHAIN KIT 50'S CHRISTMAS", 1210),
 ('JUMBO SHOPPER VINTAGE RED PAISLEY', 1202),
 ('LUNCH BAG CARS BLUE', 1197),
 ('LUNCH BAG SPACEBOY DESIGN', 1192),
 ('JAM MAKING SET PRINTED', 1182),
 ('RECIPE BOX PANTRY YELLOW DESIGN', 1180),
 ('SPOTTY BUNTING', 1172),
 ('LUNCH BAG SUKI DESIGN', 1139),
 ('ROSES REGENCY TEACUP AND SAUCER', 1138),
 ('LUNCH BAG PINK POLKADOT', 1137),
 ('WOODEN PICTURE FRAME WHITE FINISH', 1129),
 ('SET OF 4 PANTRY JE

## Most quantity sold items that is sales


In [27]:
product_grp=df.groupby(['Description'])


In [30]:
sales=product_grp['Quantity'].sum()


In [31]:
sales

Description
*Boombox Ipod Classic              1
*USB Office Mirror Ball            2
10 COLOUR SPACEBOY PEN          6392
12 COLOURED PARTY BALLOONS      2135
12 DAISY PEGS IN WOOD BOX        349
                                ... 
wrongly marked carton 22804     -256
wrongly marked. 23343 in box   -3100
wrongly sold (22719) barcode     170
wrongly sold as sets            -600
wrongly sold sets               -975
Name: Quantity, Length: 4210, dtype: int64

In [32]:
sales=pd.DataFrame(sales)
sales=sales.sort_values('Quantity',ascending=False)


In [33]:
sales

,Quantity
Description,
WORLD WAR 2 GLIDERS ASSTD DESIGNS,53847
JUMBO BAG RED RETROSPOT,47363
ASSORTED COLOUR BIRD ORNAMENT,36381
POPCORN HOLDER,36334
PACK OF 72 RETROSPOT CAKE CASES,36039
...,...
Damaged,-7540
Printing smudges/thrown away,-9058
check,-12030


In [34]:
uk_basket=df[df['Country']=='France'].groupby(['InvoiceNo','Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo')                       

In [35]:
uk_basket

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C579532,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C579562,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C580161,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
def func1(x):
    if x<=0:
        return 0
    if x>=0:
        return 1


In [37]:
uk_basket=uk_basket.applymap(func1)
#uk_basket.drop(['?lost','?display?','?missing','?sold as sets?','????damages????','????missing','???lost','???missing','?display?','??','?? missing','???','????damages????'],inplace=True,axis=1)


In [38]:
uk_basket

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537065,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537463,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C579532,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C579562,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C580161,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Most Frequent items sets 

In [39]:
freq_items=apriori(uk_basket,min_support=0.03,use_colnames=True)

In [40]:
freq_items

,support,itemsets
0,0.030369,(3 PIECE SPACEBOY COOKIE CUTTER SET)
1,0.039046,(36 PENCILS TUBE RED RETROSPOT)
2,0.060738,(4 TRADITIONAL SPINNING TOPS)
3,0.082430,(ALARM CLOCK BAKELIKE GREEN)
4,0.036876,(ALARM CLOCK BAKELIKE ORANGE)
...,...,...
480,0.030369,"(PACK OF 6 SKULL PAPER PLATES, SET/6 RED SPOTT..."
481,0.030369,"(SET/6 RED SPOTTY PAPER PLATES, POSTAGE, PACK ..."
482,0.032538,"(PACK OF 6 SKULL PAPER PLATES, POSTAGE, SET/6 ..."
483,0.045553,"(PLASTERS IN TIN WOODLAND ANIMALS, POSTAGE, PL..."


## Generating rules



In [41]:
rules=association_rules(freq_items,metric='lift',min_threshold=1)
rules.sort_values('lift',ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1221,"(SET/6 RED SPOTTY PAPER PLATES, PACK OF 20 SKU...","(PACK OF 6 SKULL PAPER PLATES, SET/6 RED SPOTT...",0.030369,0.034707,0.030369,1.000000,28.812500,0.029315,inf
1224,"(PACK OF 6 SKULL PAPER PLATES, SET/6 RED SPOTT...","(SET/6 RED SPOTTY PAPER PLATES, PACK OF 20 SKU...",0.034707,0.030369,0.030369,0.875000,28.812500,0.029315,7.757050
1237,"(SET/6 RED SPOTTY PAPER PLATES, PACK OF 6 SKUL...","(PACK OF 6 SKULL PAPER PLATES, SET/6 RED SPOTT...",0.032538,0.034707,0.030369,0.933333,26.891667,0.029239,14.479393
1220,"(SET/6 RED SPOTTY PAPER PLATES, PACK OF 6 SKUL...","(PACK OF 20 SKULL PAPER NAPKINS, SET/6 RED SPO...",0.034707,0.032538,0.030369,0.875000,26.891667,0.029239,7.739696
1225,"(PACK OF 20 SKULL PAPER NAPKINS, SET/6 RED SPO...","(SET/6 RED SPOTTY PAPER PLATES, PACK OF 6 SKUL...",0.032538,0.034707,0.030369,0.933333,26.891667,0.029239,14.479393
...,...,...,...,...,...,...,...,...,...
66,(POSTAGE),(CHILDRENS CUTLERY DOLLY GIRL),0.650759,0.060738,0.041215,0.063333,1.042738,0.001689,1.002771
376,(POSTAGE),(RETROSPOT TEA SET CERAMIC 11 PC),0.650759,0.073753,0.049892,0.076667,1.039510,0.001896,1.003156
377,(RETROSPOT TEA SET CERAMIC 11 PC),(POSTAGE),0.073753,0.650759,0.049892,0.676471,1.039510,0.001896,1.079472
361,(RED RETROSPOT CHILDRENS UMBRELLA),(POSTAGE),0.058568,0.650759,0.039046,0.666667,1.024444,0.000932,1.047722


In [42]:
def to_list(i):
    return (sorted(list(i)))

In [43]:
ma_x=rules.antecedents.apply(to_list)+ rules.consequents.apply(to_list)


In [44]:
ma_x

0                  [POSTAGE, 4 TRADITIONAL SPINNING TOPS]
1                  [4 TRADITIONAL SPINNING TOPS, POSTAGE]
2       [ALARM CLOCK BAKELIKE PINK, ALARM CLOCK BAKELI...
3       [ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...
4       [ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...
                              ...                        
1295    [POSTAGE, SET/6 RED SPOTTY PAPER CUPS, SET/20 ...
1296    [SET/6 RED SPOTTY PAPER PLATES, POSTAGE, SET/2...
1297    [SET/20 RED RETROSPOT PAPER NAPKINS, POSTAGE, ...
1298    [POSTAGE, SET/20 RED RETROSPOT PAPER NAPKINS, ...
1299    [SET/6 RED SPOTTY PAPER CUPS, POSTAGE, SET/20 ...
Length: 1300, dtype: object

In [45]:
ma_x=ma_x.apply(sorted)


In [46]:
ma_x

0                  [4 TRADITIONAL SPINNING TOPS, POSTAGE]
1                  [4 TRADITIONAL SPINNING TOPS, POSTAGE]
2       [ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...
3       [ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...
4       [ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...
                              ...                        
1295    [POSTAGE, SET/20 RED RETROSPOT PAPER NAPKINS, ...
1296    [POSTAGE, SET/20 RED RETROSPOT PAPER NAPKINS, ...
1297    [POSTAGE, SET/20 RED RETROSPOT PAPER NAPKINS, ...
1298    [POSTAGE, SET/20 RED RETROSPOT PAPER NAPKINS, ...
1299    [POSTAGE, SET/20 RED RETROSPOT PAPER NAPKINS, ...
Length: 1300, dtype: object

In [47]:
rules_sets=list(ma_x)
unique_rules_set=[list(m) for m in set(tuple(i) for i in rules_sets)]

In [48]:
index_rules=[]
for i in unique_rules_set:
    index_rules.append(rules_sets.index(i))


In [50]:
rules_no_redudancy=rules.iloc[index_rules,:]
rules_no_redudancy

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
426,(POSTAGE),(WOODLAND MINI BACKPACK),0.650759,0.034707,0.030369,0.046667,1.344583,0.007783,1.012545
308,(STRAWBERRY LUNCH BOX WITH CUTLERY),(PLASTERS IN TIN CIRCUS PARADE),0.104121,0.143167,0.039046,0.375000,2.619318,0.024139,1.370933
768,"(PACK OF 6 SKULL PAPER PLATES, PACK OF 6 SKULL...",(PACK OF 20 SKULL PAPER NAPKINS),0.043384,0.041215,0.032538,0.750000,18.197368,0.030750,3.835141
840,"(SET/6 RED SPOTTY PAPER PLATES, POSTAGE)",(PACK OF 6 SKULL PAPER CUPS),0.091106,0.054230,0.030369,0.333333,6.146667,0.025428,1.418655
198,(ROUND SNACK BOXES SET OF4 WOODLAND),(LUNCH BAG SPACEBOY DESIGN),0.134490,0.101952,0.032538,0.241935,2.373027,0.018826,1.184659
...,...,...,...,...,...,...,...,...,...
78,(POSTAGE),(CIRCUS PARADE LUNCH BOX),0.650759,0.058568,0.045553,0.070000,1.195185,0.007439,1.012292
280,(POSTAGE),(PACK OF 60 MUSHROOM CAKE CASES),0.650759,0.032538,0.030369,0.046667,1.434222,0.009194,1.014820
330,(POSTAGE),(PLASTERS IN TIN VINTAGE PAISLEY),0.650759,0.032538,0.030369,0.046667,1.434222,0.009194,1.014820
666,"(PLASTERS IN TIN SPACEBOY, LUNCH BAG SPACEBOY ...",(PLASTERS IN TIN CIRCUS PARADE),0.045553,0.143167,0.032538,0.714286,4.989177,0.026016,2.998915


In [51]:
rules_no_redudancy=rules_no_redudancy.sort_values('lift',ascending=False)
rules_no_redudancy

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
102,(DOLLY GIRL MINI BACKPACK),(WOODLAND MINI BACKPACK),0.034707,0.034707,0.030369,0.875000,25.210937,0.029164,7.722343
86,(DOLLY GIRL CHILDRENS CUP),(DOLLY GIRL CHILDRENS BOWL),0.034707,0.039046,0.032538,0.937500,24.010417,0.031183,15.375271
792,"(SET/6 RED SPOTTY PAPER PLATES, PACK OF 6 SKUL...",(PACK OF 20 SKULL PAPER NAPKINS),0.034707,0.041215,0.030369,0.875000,21.230263,0.028938,7.670282
90,(SPACEBOY CHILDRENS BOWL),(DOLLY GIRL CHILDRENS BOWL),0.043384,0.039046,0.034707,0.800000,20.488889,0.033013,4.804772
252,(PACK OF 6 SKULL PAPER PLATES),(PACK OF 20 SKULL PAPER NAPKINS),0.047722,0.041215,0.036876,0.772727,18.748804,0.034909,4.218655
...,...,...,...,...,...,...,...,...,...
70,(POSTAGE),(CHILDRENS CUTLERY SPACEBOY),0.650759,0.058568,0.041215,0.063333,1.081358,0.003101,1.005087
404,(POSTAGE),(SKULL LUNCH BOX WITH CUTLERY),0.650759,0.043384,0.030369,0.046667,1.075667,0.002136,1.003443
66,(POSTAGE),(CHILDRENS CUTLERY DOLLY GIRL),0.650759,0.060738,0.041215,0.063333,1.042738,0.001689,1.002771
376,(POSTAGE),(RETROSPOT TEA SET CERAMIC 11 PC),0.650759,0.073753,0.049892,0.076667,1.039510,0.001896,1.003156


In [52]:
rules2=rules_no_redudancy.iloc[:,:2]
rules2

,antecedents,consequents
102,(DOLLY GIRL MINI BACKPACK),(WOODLAND MINI BACKPACK)
86,(DOLLY GIRL CHILDRENS CUP),(DOLLY GIRL CHILDRENS BOWL)
792,"(SET/6 RED SPOTTY PAPER PLATES, PACK OF 6 SKUL...",(PACK OF 20 SKULL PAPER NAPKINS)
90,(SPACEBOY CHILDRENS BOWL),(DOLLY GIRL CHILDRENS BOWL)
252,(PACK OF 6 SKULL PAPER PLATES),(PACK OF 20 SKULL PAPER NAPKINS)
...,...,...
70,(POSTAGE),(CHILDRENS CUTLERY SPACEBOY)
404,(POSTAGE),(SKULL LUNCH BOX WITH CUTLERY)
66,(POSTAGE),(CHILDRENS CUTLERY DOLLY GIRL)
376,(POSTAGE),(RETROSPOT TEA SET CERAMIC 11 PC)


In [58]:
!pip install fuzzywuzzy



You should consider upgrading via the 'c:\users\dpunyamu\anaconda3\python.exe -m pip install --upgrade pip' command.


In [59]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

rules2['antecedents']=rules2['antecedents'].apply(list)
rules2['consequents']=rules2['consequents'].apply(list)

C:\Users\dpunyamu\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [60]:
rules3=rules2['antecedents']+rules2['consequents']
rules3=pd.DataFrame(rules3)
rules3.columns=['rules_list']

In [62]:
c=['WOODLAND MINI BACKPACK']


In [63]:
b=[i for i in rules3['rules_list']]
b[1]


['DOLLY GIRL CHILDRENS CUP', 'DOLLY GIRL CHILDRENS BOWL']

In [64]:

a=fuzz.ratio(c,b[5])

In [65]:
a

16

In [73]:
list89=[]
i=0 
while i < (len(b)-1):
    d=b[i]
    a=fuzz.ratio(c,d)
    list89.append(a)   
    i+=1

In [74]:
d

['POSTAGE', 'RETROSPOT TEA SET CERAMIC 11 PC']

In [75]:
d

['POSTAGE', 'RETROSPOT TEA SET CERAMIC 11 PC']

In [76]:
list89

[32,
 33,
 30,
 36,
 24,
 22,
 19,
 25,
 17,
 33,
 25,
 23,
 20,
 25,
 32,
 30,
 28,
 23,
 27,
 22,
 27,
 33,
 29,
 25,
 35,
 21,
 26,
 35,
 30,
 36,
 35,
 31,
 26,
 22,
 31,
 19,
 20,
 26,
 19,
 16,
 32,
 25,
 28,
 25,
 33,
 31,
 16,
 41,
 19,
 36,
 33,
 20,
 20,
 20,
 20,
 21,
 38,
 22,
 33,
 49,
 27,
 23,
 27,
 22,
 31,
 22,
 25,
 21,
 34,
 23,
 22,
 18,
 17,
 40,
 27,
 50,
 25,
 30,
 22,
 39,
 24,
 21,
 34,
 51,
 50,
 26,
 30,
 44,
 25,
 38,
 39,
 26,
 62,
 27,
 56,
 50,
 19,
 25,
 33,
 69,
 27,
 54,
 26,
 54,
 28,
 37,
 23,
 51,
 31,
 70,
 63,
 41,
 47,
 24,
 24,
 29,
 25,
 26,
 34,
 58,
 26,
 27,
 20,
 31,
 19,
 65,
 22,
 51,
 23,
 24,
 31,
 31,
 28,
 43,
 27,
 27,
 44,
 58,
 56,
 46,
 24,
 51,
 50,
 23,
 22,
 54,
 68,
 24,
 35,
 53,
 69,
 22,
 58,
 53,
 41,
 51,
 27,
 19,
 19,
 17,
 25,
 26,
 62,
 69,
 62,
 58,
 51,
 32,
 23,
 19,
 58,
 44,
 50,
 70,
 58,
 64,
 70,
 22,
 67,
 21,
 72,
 55,
 31,
 50,
 46,
 25,
 35,
 63,
 49,
 18,
 53,
 58,
 48,
 72,
 64,
 22,
 65,
 63,
 42,
 38,
